In [1]:
!git config --global credential.helper store
!pip install accelerate
!git config --global user.name "Neelectric"
!git config --global user.email "Neel.R@web.de"

In [2]:
token = input(f"Enter token: ")

In [ ]:
from src.biollama

In [3]:
print("started script")
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
print("imported...")

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf", 
                                          token = token,
                                          cache_dir = "../hf_cache/")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-13b-chat-hf", 
                                             token = token,
                                             device_map = "auto",
                                             torch_dtype = torch.float16,
                                             cache_dir = "../hf_cache/")

print("model loaded!")

started script


There was a problem when trying to write in your cache folder (/root/.cache/huggingface/hub). You should set the environment variable TRANSFORMERS_CACHE to a writable directory.


imported...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

model loaded!


In [5]:
from time import time
max_new_tokens = 400
prompt = "In the era of generative AI, "
medmcqa2 = """
Question.
Low insulin to glucagon ratio is not seen in:
(A) Glycogen synthesis
(B) Glycogen breakdown
(C) Gluconeogenesis
(D) Ketogenesis
Answer. 
Let's think step by step. To this end, let's first reflect on the question and which concepts will be relevant in answering it. Then, let's brainstorm on how concepts in the answer options relate back to it. Finally, let's evaluate each option in relation to the question, choosing one.
"""
tokenized_prompt = tokenizer.encode(medmcqa2, return_tensors = "pt").to('cuda')
time_before = time()
raw_output = model.generate(tokenized_prompt,
                            # max_new_tokens = max_new_tokens,
                            temperature = 0.01)
time_after = time()
time_taken = time_after - time_before
untokenized_output = tokenizer.decode(raw_output[0], skip_special_tokens = True)
num_generated = len(raw_output[0]) - len(tokenized_prompt[0])
print(untokenized_output)
print(f"newly generated {num_generated}")
print(f"{num_generated / time_taken} t/s")


Question.
Low insulin to glucagon ratio is not seen in:
(A) Glycogen synthesis
(B) Glycogen breakdown
(C) Gluconeogenesis
(D) Ketogenesis
Answer. 
Let's think step by step. To this end, let's first reflect on the question and which concepts will be relevant in answering it. Then, let's brainstorm on how concepts in the answer options relate back to it. Finally, let's evaluate each option in relation to the question, choosing one.

Answer: C

Explanation:

The question asks about low insulin to glucagon ratio, which is relevant to glucose metabolism. Glucagon is a hormone that raises blood glucose levels by stimulating glycogen breakdown and gluconeogenesis. Insulin, on the other hand, lowers blood glucose levels by promoting glycogen synthesis and inhibiting gluconeogenesis.

Now, let's evaluate each answer option in relation to the question:

A) Glycogen synthesis: Insulin promotes glycogen synthesis, so a low insulin to glucagon ratio would not be associated with glycogen synthesis.

In [6]:
medmcqa = """
Question:
Axonal transport is:
(A) Antegrade
(B) Retrograde
(C) Antegrade and retrograde
(D) None
Answer: Let's reflect on each response individually, without choosing whether it is correct yet. Then, discuss the responses in relation to the question, hypothesizing on whether it might be the right answer. Finally choose one from these candidates after considering them carefully.
"""